In [1]:
import re
import numpy as np
from collections import Counter

PBIL Algorithm to solve weighted-Max-2-SAT problems

Boolean formula is assumed to be in Conjunctive Normal Form (CNF) over variables $x_1,x_2, \ldots x_N$

CNF $ (x_1 \vee \neg x_1) \wedge (x_2 \vee x_3) \wedge (\neg x_2 \vee x_4) $ is represented as the string "1!123!24" which is obtained by concatenating the variable $x_i$ as $i$ in left to right order, using "!" to represent negation

In [2]:
def get_clauses(formula):
    return re.findall(r'!?\d!?\d',formula)

def number_variables(formula):
    return len(set(re.findall(r'\d',formula)))

# Evaluate a clause where each literal is a 0 or a 1
def evaluate_clause(clause):
    is_negated = False
    result = False
    for x in clause:
        if x == '!':
            is_negated = True
            continue
        x = bool(int(x))
        if is_negated:
            result = result or not(x)
            is_negated = False
        else:
            result = result or x
    return result

In [3]:
# Sample probability distribution
# solution[i]: The value of variable i+1
def sample(P):
    n = P.size
    solution = np.empty(n,dtype=int)
    for i in range(n):
        solution[i] = np.random.choice([0,1],size=None,p=[1-P[i],P[i]])
    return solution

# Compute the empirical probabilities (probability of each variable being a 1 over the samples)
def empirical_probabilities(samples):
    # Number of variables
    n = samples[0].size
    p = np.empty(n)
    N = len(samples)
    for i in range(n):
        values_i = [sample[i] for sample in samples]
        count = Counter(values_i)
        p[i] = count[1] / N
    return p

def update(alpha, P, empirical_probabilities):
    return alpha * P + (1-alpha) * empirical_probabilities

# Returns whether the distribution is valid (probabilities are between 0 and 1)
def check_P(P):
    return not(np.any(model.P < 0)) and not(np.any(model.P > 1))

# Evaluate the fitness: sum of the weights of the clauses that are satisfied
# weights: The weights corresponding to each clause
def fitness(formula, solution, weights):
    clauses = get_clauses(formula)
    fitness = 0
    for clause, weight in zip(clauses, weights):
        # Substitute variables with its value stored in solution
        clause = re.sub(r'\d', lambda m: _sub(solution,m), clause)
        if evaluate_clause(clause):
            fitness += weight
    return fitness

def _sub(solution,match):
    i = int(match.group(0)) - 1
    return str(solution[i])

In [4]:
class Model:
    def __init__(self, formula, weights):
        self.formula = formula
        self.weights = weights
        self.clauses = get_clauses(formula)
        self.N = number_variables(formula)
        # Each entry is the probability of a variable taking value 1
        self.P = np.full(self.N, 0.5)
        self.P0 = self.P.copy()
    
# N: Number of iterations
# Q: Population Size
# Q1: Sample Size (Number of fit samples used to update probabilities)
#alpha: Learning rate between 0 and 1
def  main(model, N, Q, Q1, alpha):
    for n in range(N):
        # Generate samples from the current distribution
        samples = [sample(model.P) for _ in range(Q)]
        samples.sort(key=lambda x: fitness(formula,x, model.weights), reverse=True)
        # Use the fittest samples to update distribution
        samples = samples[:Q1]
        empirical_p = empirical_probabilities(samples)
        model.P = update(alpha, model.P, empirical_p)

In [5]:
# (x1+x2)(x1+!x2)(!x1+x2)(!x1+!x2)
# Not satisfiable: Can only make 3 clauses satisfiable

formula = '121!2!12!1!2'
weights = [1,2,1,1]

model = Model(formula, weights)

N = 200
Q = 100
Q1 = 20
alpha = 0.7

main(model,N,Q,Q1,alpha)

In [14]:
sol = sample(model.P)
model.P, sol, fitness(model.formula, sol, model.weights)

(array([1.00000000e+00, 6.76508174e-11]), array([1, 0]), 4)